# DoIT Schedule Building
### Ray Smith - ECE 524

In [1]:
ENV["GUROBI_HOME"] = "/Library/gurobi911/mac64"
ENV["GRB_LICENSE_FILE"] = "/Library/gurobi911/gurobi.lic"
import Pkg

In [2]:
# Imports
using JuMP
Pkg.add("Gurobi")
Pkg.build("Gurobi")
using Gurobi

using CSV
using JSON
using DataFrames
using OrderedCollections

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


    Fetching: [============

    Fetching:

Fetching: [========================================>]  100.0 %               ]  62.1 %===============================>         ]  77.2 %

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
   Building Gurobi → `~/.julia/packages/Gurobi/JcjAE/deps/build.log`
┌ Info: Precompiling Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
└ @ Base loading.jl:1278


In [3]:
# Display Settings

ENV["COLUMNS"]=120
ENV["LINES"] = 32

32

In [42]:
# Schedule Rules

# General Schedule Layout
hours = 1:32
hours_real = 7:0.5:22.5
days = 1:7

# Shift Types and Training Requirements
shifts = ["HDQA", "Phones", "Chat/Email", "Walk-In"]
training_levels = ["Pick 1", "Walk-In", "Pick 3", "Chat/Email", "HDQA", "SLP"]
requirements = OrderedDict()
requirements[shifts[1]] = [training_levels[5]]
requirements[shifts[2]] = [training_levels[1]]
requirements[shifts[3]] = [training_levels[4]]
requirements[shifts[4]] = [training_levels[2]]

# Allowed Flexibility in (Extra) Agents Scheduled
flexibility = OrderedDict()
flexibility[shifts[1]] = 0
flexibility[shifts[2]] = 2
flexibility[shifts[3]] = 0
flexibility[shifts[4]] = 1

# Other Rules
adv_phones_req = 1 # min number of pick 3 trained agents scheduled over a phones shift
min_shift_length = 4 # in 1/2 hour blocks
agent_min = 30 # in 1/2 hour blocks
agent_max = 50 # in 1/2 hour blocks
sch_yellow = true
min_hdqa_shifts = 8
max_hdqa_shifts = 24

24

In [5]:
# Data Input

# Schedule Shifts
raw_week = CSV.read(joinpath(@__DIR__, "week_req.csv"), DataFrame)
week_shift_types = names(raw_week)
week = Matrix(raw_week)
raw_weekend = CSV.read(joinpath(@__DIR__, "weekend_req.csv"), DataFrame)
weekend_shift_types = names(raw_weekend)
weekend = Matrix(raw_weekend)

weekday = [OrderedDict() for i in hours]
weekendday = [OrderedDict() for i in hours]
for i in hours
    j = 1
    for shift in week_shift_types
        weekday[i][shift] = week[i, j]
        j += 1
    end
    k = 1
    for shift in weekend_shift_types
        weekendday[i][shift] = weekend[i, k]
        k += 1
    end
end

# Availability
string_data = join(readlines("availability.json"))
raw_avail = JSON.parse(string_data)
agents = keys(raw_avail)
avail = OrderedDict()
for agent in agents
    raw_agent_avail = raw_avail[agent]
    agent_avail = OrderedDict()
    for i in hours
        agent_avail[i] = raw_agent_avail[i]
    end
    avail[agent] = agent_avail
end

# Training
string_data = join(readlines("agent_training.json"))
raw_trainings = JSON.parse(string_data)
all_agents = keys(raw_trainings)
trainings = OrderedDict()
for agent in all_agents
    raw_agent_trainings = raw_trainings[agent]
    base_job = raw_agent_trainings[:"Base"]
    agent_trainings = raw_agent_trainings[:"Trainings"]
    if startswith(base_job, "SLP")
        append!(agent_trainings, ["SLP"])
    end
    trainings[agent] = agent_trainings
end

In [22]:
# Model Helper Functions

# Check if agent has training
function has_training(agent, training_level)
    return training_level in trainings[agent]
end

# Check if agent meets all training requirements for a shift
function meets_requirements(agent, shift)
    shift_req = requirements[shift]
    for training in shift_req
        if !has_training(agent, training)
            return false
        end
    end
    return true
end

# Check if agent is available (green/yellow) at a day/time
function shift_available(agent, day, time)
    agent_avail = avail[agent]
    time_slice = agent_avail[time]
    color = time_slice[day]
    
    if color == "G" || (color == "Y" && sch_yellow)
        return true
    else
        return false
    end
end

# Get shift types for a day
function get_shift_types(day)
    if (day == 1 || day == 7)
        return weekend_shift_types
    else
        return week_shift_types
    end
end

# Get shift requirements for a day
function get_day_req(day)
    if (day == 1 || day == 7)
        return weekendday
    else
        return weekday
    end
end

# Get agent total available hours
function get_total_hours(agent)
    total = 0
    for day in days
        for hour in hours
            if (shift_available(agent, day, hour))
                total += 1
            end
        end
    end
    return total
end

# Get agent min hours
function get_min_hours(agent)
    return min(get_total_hours(agent), agent_min)
end

# Get neighboring shifts
function get_neighbor_shifts(time)
    range = []
    for hour in hours
        if abs(hour - time) < min_shift_length
            append!(range, hour)
        end
    end
    return range
end

# Get max workers allowed for a shift at a time
function get_max_allowed(shift, day, time)
    shift_types = get_shift_types(day)
    day_req = get_day_req(day)
    
    base_needed = day_req[time][shift]
    if base_needed == 0
        return 0
    else
        return base_needed + flexibility[shift]
    end
end

get_max_allowed (generic function with 1 method)

##### Model 1: Just assigned work or not

In [7]:
# Scheduling Model Setup

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

@variable(m, sch[agents, days, hours], Bin) # If agent scheduled
@variable(m, req[agents, shifts], Bin) # If agent meets shift requirement
@variable(m, adv[agents], Bin) # If agent is adv. phones trained

# Set shift eligibility
for agent in agents
    for shift in shifts
        @constraint(m, req[agent, shift] == meets_requirements(agent, shift))
    end
    @constraint(m, adv[agent] == has_training(agent, "Pick 3"))
end

# Shift requirement constraints
for day in days
    shift_types = get_shift_types(day)
    day_req = get_day_req(day)
    
    for hour in hours
        max_workers = 0
        min_workers = 0
        for shift in shift_types
            max_workers += day_req[hour][shift] + flexibility[shift]
            min_workers += day_req[hour][shift]
            
            # Ensure enough trained agents on each shift
            @constraint(m, sum(sch[agent, day, hour] * req[agent, shift] for agent in agents) >= day_req[hour][shift])
            
            # Ensure enough total agents on each shift
            @constraint(m, sum(sch[agent, day, hour] for agent in agents) >= min_workers)
        end
        
        # Advanced phone agent requirement
        @constraint(m, sum(sch[agent, day, hour] * adv[agent] for agent in agents) >= adv_phones_req)
        
        # Ensure no excessive scheduling of extra workers
        @constraint(m , sum(sch[agent, day, hour] for agent in agents) <= max_workers)
    end
end

# Ensure agents only scheduled when green/yellow
for agent in agents
    for day in days
        for hour in hours
            @constraint(m, sch[agent, day, hour] <= shift_available(agent, day, hour))
        end
    end
end

# Ensure each agent works within the min/max allowed hour range
for agent in agents
    @constraint(m, sum(sch[agent, day, hour] for day in days, hour in hours) >= get_min_hours(agent))
    @constraint(m, sum(sch[agent, day, hour] for day in days, hour in hours) <= agent_max)
end

# Don't allow shifts of < minimum duration
for agent in agents
    for day in days
        for hour in hours
            @constraint(m, sum(sch[agent, day, h] for h in get_neighbor_shifts(hour)) >= sch[agent, day, hour] * min_shift_length)
        end
    end
end

# Schedule the minimum amount of agents possible
@objective(m, Min, sum(sch))

# Solve the scheduling problem
optimize!(m)
println("Total Hours Scheduled: ", sum(value.(sch)))
# 7542 is max using all green availability
# 12092 is max using all green/yellow availability


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-05-06
Total Hours Scheduled: 1445.0


In [8]:
# View agent schedule

X = value.(sch)
X["RAYS",:,:]

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:7
    Dimension 2, 1:32
And data, a 7×32 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [9]:
# Show Daily Schedule Nicely

schedule = OrderedDict()
for day in days
    schedule[day] = OrderedDict()
    for hour in hours
        schedule[day][hours_real[hour]] = []
        for agent in agents
            if X[agent, day, hour] == 1
                append!(schedule[day][hours_real[hour]], [agent])
            end
        end
    end
end

day_to_show = 2 # Monday
for hour in hours_real
    println(hour, " : ", schedule[day_to_show][hour])
end

7.0 : Any["HUNT", "JOBA", "BECK", "TBOS"]
7.5 : Any["HUNT", "JOBA", "BECK", "TBOS"]
8.0 : Any["OCLA", "DBON", "BECK", "CKLE", "MJMO", "JEMA"]
8.5 : Any["OCLA", "DBON", "CKLE", "MJMO", "JLHE", "JEMA"]
9.0 : Any["OCLA", "ALZA", "DBON", "DRDO", "VRNA", "JESO", "MJMO", "JLHE", "JEMA", "DUNN"]
9.5 : Any["KTEM", "OCLA", "ALZA", "PAKA", "DRDO", "VRNA", "JESO", "JLHE", "DUNN", "NINA"]
10.0 : Any["KTEM", "STEF", "ELLI", "PAKA", "DRDO", "JESO", "TBOS", "GOFF", "MJMO", "RWAS", "NINA", "JBRI"]
10.5 : Any["KTEM", "STEF", "ELLI", "DRDO", "BECK", "JESO", "TBOS", "GOFF", "MJMO", "RWAS", "GUZM", "JBRI"]
11.0 : Any["FOER", "STEF", "BECK", "MEZI", "LBAN", "BING", "PETU", "JESO", "MUSA", "RWAS", "GUZM", "JBRI"]
11.5 : Any["FOER", "KTEM", "JOBA", "HAYA", "MEZI", "LBAN", "BING", "PETU", "ADWI", "MUSA", "GARC", "GENI"]
12.0 : Any["FOER", "KTEM", "ALZA", "STEF", "JOBA", "HAYA", "MEZI", "LKRE", "ADWI", "JEMA", "GARC", "GENI"]
12.5 : Any["MORI", "ALZA", "STEF", "JOBA", "MEZI", "LKRE", "ADWI", "HOST", "JLHE", "J

##### Model 2: Assigns Roles

In [50]:
# Role assigning model

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

@variable(m, sch[agents, days, hours, shifts], Bin) # If agent scheduled and for what role

# Only allow each agent to be scheduled for 1 shift at a time
for agent in agents
    for day in days
        for hour in hours
            @constraint(m, sum(sch[agent, day, hour, shift] for shift in shifts) <= 1)
        end
    end
end

# Only allow agents to be scheduled when green/yellow
for agent in agents
    for day in days
        for hour in hours
            for shift in shifts
                @constraint(m, sch[agent, day, hour, shift] <= shift_available(agent, day, hour))
            end
        end
    end
end

# Shift requirement constraints
for day in days
    shift_types = get_shift_types(day)
    day_req = get_day_req(day)
    
    for hour in hours
        for shift in shift_types
            # Ensure enough trained agents on each shift
            @constraint(m, sum(sch[agent, day, hour, shift] * meets_requirements(agent, shift) for agent in agents) >= day_req[hour][shift])
            
            # Ensure no excessive scheduling of extra workers
            @constraint(m, sum(sch[agent, day, hour, shift] for agent in agents) <= get_max_allowed(shift, day, hour))
        end
        
        # Advanced phone agent requirement
        @constraint(m, sum(sch[agent, day, hour, "Phones"] * has_training(agent, "Pick 3") for agent in agents) >= adv_phones_req)
    end
end

# Don't allow shifts of < minimum duration
for agent in agents
    for day in days
        for hour in hours
            for shift in shifts
                @constraint(m, sum(sch[agent, day, h, shift] for h in get_neighbor_shifts(hour)) >= sch[agent, day, hour, shift] * min_shift_length)
            end
        end
    end
end

# Ensure each agent works within the min/max allowed hour range
for agent in agents
    @constraint(m, sum(sch[agent, day, hour, shift] for day in days, hour in hours, shift in shifts) >= get_min_hours(agent))
    @constraint(m, sum(sch[agent, day, hour, shift] for day in days, hour in hours, shift in shifts) <= agent_max)
end

# Minimum HDQA Shift Requirement
for agent in agents
    @constraint(m, sum(sch[agent, day, hour, "HDQA"] for day in days, hour in hours) >= min_hdqa_shifts * has_training(agent, "HDQA"))
    @constraint(m, sum(sch[agent, day, hour, "HDQA"] for day in days, hour in hours) <= max_hdqa_shifts * has_training(agent, "HDQA"))
end

# No SLPs on Phones or Chat/Email Shifts
for agent in agents
    for day in days
        for hour in hours
            @constraint(m, sch[agent, day, hour, "Phones"] <= !has_training(agent, "SLP"))
            @constraint(m, sch[agent, day, hour, "Chat/Email"] <= !has_training(agent, "SLP"))
        end
    end
end

# Schedule the minimum amount of agents possible
@objective(m, Min, sum(sch))

# Solve the scheduling problem
optimize!(m)
println("Total Hours Scheduled: ", sum(value.(sch)))
# 7542 is max using all green availability
# 12092 is max using all green/yellow availability


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-05-06
Total Hours Scheduled: 2218.0


In [51]:
# Show Daily Schedule Nicely

X = value.(sch)

schedule = OrderedDict()
for day in days
    shift_types = get_shift_types(day)
    schedule[day] = OrderedDict()
    for shift in shift_types
        schedule[day][shift] = OrderedDict()
        for hour in hours
            schedule[day][shift][hour] = []
            for agent in agents
                if X[agent, day, hour, shift] == 1
                    append!(schedule[day][shift][hour], [agent])
                end
            end
        end
    end
end

day_to_show = 2
shift_types = get_shift_types(day_to_show)
data = schedule[day_to_show]
cols = []
for shift in shift_types
    col = []
    for hour in hours
        append!(col, [data[shift][hour]])
    end
    append!(cols, [col])
end
if length(shift_types) == 4
    df = DataFrame(time = hours_real, hdqa = cols[1], phones = cols[2], chat = cols[3], walkin = cols[4])
elseif length(shift_types) == 2
    df = DataFrame(time = hours_real, hdqa = cols[1], phones = cols[2])
end

df

,time,hdqa,phones,chat,walkin
,Float64,Any,Any,Any,Any
1,7.0,"[""TBOS""]","[""BECK"", ""LBAN"", ""BING""]",[],[]
2,7.5,"[""TBOS""]","[""BECK"", ""LBAN"", ""BING""]",[],[]
3,8.0,"[""TBOS""]","[""JOBA"", ""BECK"", ""LBAN"", ""BING"", ""NINA""]","[""GENI"", ""YFER""]",[]
4,8.5,"[""TBOS""]","[""JOBA"", ""BECK"", ""LBAN"", ""BING"", ""NINA""]","[""GENI"", ""YFER""]",[]
5,9.0,"[""THOK""]","[""JOBA"", ""LBAN"", ""MJMO"", ""GUZM"", ""NINA""]","[""GENI"", ""YFER""]","[""ALZA"", ""BCON"", ""RMAJ""]"
6,9.5,"[""THOK""]","[""JOBA"", ""JESO"", ""MJMO"", ""RWAS"", ""GUZM"", ""NINA""]","[""GENI"", ""YFER""]","[""ALZA"", ""BCON"", ""RMAJ""]"
7,10.0,"[""THOK""]","[""JESO"", ""MUSA"", ""MJMO"", ""RWAS"", ""GUZM"", ""JBRI""]","[""GENI"", ""JLEE"", ""YFER""]","[""ALZA"", ""BCON"", ""RMAJ""]"
8,10.5,"[""THOK""]","[""JESO"", ""MUSA"", ""MJMO"", ""RWAS"", ""GUZM"", ""JBRI""]","[""GENI"", ""JLEE"", ""YFER""]","[""ALZA"", ""BCON"", ""RMAJ""]"
9,11.0,"[""DUNN""]","[""FOER"", ""JESO"", ""MUSA"", ""RWAS"", ""GUZM"", ""JBRI""]","[""GENI"", ""JLEE"", ""YFER""]","[""ALZA"", ""BCON"", ""HEER""]"
